In [1]:
import os
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Reshape, BatchNormalization, Dropout
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from sklearn.model_selection import train_test_split
from tqdm import tqdm

# Set paths for training and validation
TRAIN_PATH = '../DeepGlobe Dataset/train_subSub1'
VALID_PATH = '../DeepGlobe Dataset/valid_subSub'

# Parameters
PATCH_SIZE = 64  # Size of the patches
IMG_SIZE = (256, 256)  # Resized image size for input to the network
BATCH_SIZE = 16
EPOCHS = 50

# Function to load images and their corresponding masks
def load_images_from_directory(directory, image_size=IMG_SIZE):
    images, masks = [], []
    for filename in tqdm(os.listdir(directory)):
        if 'sat' in filename:
            # Load satellite image
            sat_img = load_img(os.path.join(directory, filename), target_size=image_size)
            sat_img = img_to_array(sat_img) / 255.0  # Normalize
            images.append(sat_img)
            
            # Load corresponding mask image
            mask_filename = filename.replace('sat.jpg', 'mask.png')
            mask_img = load_img(os.path.join(directory, mask_filename), target_size=image_size, color_mode="grayscale")
            mask_img = img_to_array(mask_img) / 255.0  # Normalize to 0-1
            masks.append(mask_img)
    return np.array(images), np.array(masks)

# Function to create patches from images and masks
def create_patches(images, masks, patch_size=PATCH_SIZE):
    img_patches, mask_patches = [], []
    for img, mask in zip(images, masks):
        for i in range(0, IMG_SIZE[0], int(patch_size)):
            for j in range(0, IMG_SIZE[1], patch_size):
                img_patch = img[i:i+patch_size, j:j+patch_size]
                mask_patch = mask[i:i+patch_size, j:j+patch_size]
                img_patches.append(img_patch)
                mask_patches.append(mask_patch)
    return np.array(img_patches), np.array(mask_patches)

# Load the training data
train_images, train_masks = load_images_from_directory(TRAIN_PATH)

# Split into patches
train_img_patches, train_mask_patches = create_patches(train_images, train_masks)

# Build a traditional CNN for image segmentation
def build_cnn(input_shape=(PATCH_SIZE, PATCH_SIZE, 3)):
    model = Sequential()
    
    # First convolutional block
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same', input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    # Second convolutional block
    model.add(Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    # Third convolutional block
    model.add(Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    # Fourth convolutional block (optional for even deeper network)
    model.add(Conv2D(512, kernel_size=(3, 3), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    # Add a dropout layer to prevent overfitting
    model.add(Dropout(0.5))
    
    # Fully connected layers
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    
    # Output layer reshaped to match the mask size (patch_size x patch_size x 1)
    model.add(Dense(PATCH_SIZE * PATCH_SIZE, activation='sigmoid'))
    model.add(Reshape((PATCH_SIZE, PATCH_SIZE, 1)))
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

# Build the model
model = build_cnn(input_shape=(PATCH_SIZE, PATCH_SIZE, 3))
model.summary()

# Train the model
model.fit(train_img_patches, train_mask_patches, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_split=0.1)

# Function to predict masks for the validation set
def predict_and_save_masks(model, valid_dir, output_dir, image_size=IMG_SIZE, patch_size=PATCH_SIZE):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    for filename in os.listdir(valid_dir):
        if 'sat' in filename:
            # Load the validation satellite image
            sat_img = load_img(os.path.join(valid_dir, filename), target_size=image_size)
            sat_img = img_to_array(sat_img) / 255.0  # Normalize
            sat_patches = create_patches([sat_img], [sat_img])[0]
            
            # Predict mask for each patch
            pred_patches = model.predict(sat_patches)
            
            # Reconstruct full-size mask from patches
            pred_mask = np.zeros(image_size, dtype=np.float32)
            patch_index = 0
            for i in range(0, image_size[0], patch_size):
                for j in range(0, image_size[1], patch_size):
                    pred_mask[i:i+patch_size, j:j+patch_size] = pred_patches[patch_index].reshape(patch_size, patch_size)
                    patch_index += 1
            
            # Save predicted mask
            output_filename = filename.replace('sat.jpg', 'predicted_mask.png')
            cv2.imwrite(os.path.join(output_dir, output_filename), (pred_mask * 255).astype(np.uint8))

# Predict and save masks for validation images
VALID_OUTPUT_DIR = 'predicted_masks2'
predict_and_save_masks(model, VALID_PATH, VALID_OUTPUT_DIR)

100%|██████████| 204/204 [00:02<00:00, 74.25it/s]
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 64, 64, 64)     │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 64, 64, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 32, 32, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 32, 32, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 32, 32, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 16, 16, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 16, 16, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 16, 16, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 8, 8, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 8, 8, 512)      │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 8, 8, 512)      │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 4, 4, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 4, 4, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 8192)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │     4,194,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4096)           │     2,101,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape (Reshape)               │ (None, 64, 64, 1)      │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,850,880 (29.95 MB)

 Trainable params: 7,848,960 (29.94 MB)

 Non-trainable params: 1,920 (7.50 KB)

Epoch 1/50
92/92 ━━━━━━━━━━━━━━━━━━━━ 29s 289ms/step - accuracy: 0.8938 - loss: 0.3039 - val_accuracy: 0.9598 - val_loss: 0.2282
Epoch 2/50
92/92 ━━━━━━━━━━━━━━━━━━━━ 25s 270ms/step - accuracy: 0.9611 - loss: 0.1891 - val_accuracy: 0.9598 - val_loss: 0.1865
Epoch 3/50
92/92 ━━━━━━━━━━━━━━━━━━━━ 22s 244ms/step - accuracy: 0.9612 - loss: 0.1883 - val_accuracy: 0.9598 - val_loss: 0.5704
Epoch 4/50
92/92 ━━━━━━━━━━━━━━━━━━━━ 16s 171ms/step - accuracy: 0.9602 - loss: 0.1872 - val_accuracy: 0.9598 - val_loss: 0.1769
Epoch 5/50
92/92 ━━━━━━━━━━━━━━━━━━━━ 17s 186ms/step - accuracy: 0.9646 - loss: 0.1659 - val_accuracy: 0.9598 - val_loss: 0.1771
Epoch 6/50
92/92 ━━━━━━━━━━━━━━━━━━━━ 17s 183ms/step - accuracy: 0.9635 - loss: 0.1729 - val_accuracy: 0.9598 - val_loss: 0.1689
Epoch 7/50
92/92 ━━━━━━━━━━━━━━━━━━━━ 16s 171ms/step - accuracy: 0.9616 - loss: 0.1729 - val_accuracy: 0.9598 - val_loss: 0.1706
Epoch 8/50
92/92 ━━━━━━━━━━━━━━━━━━━━ 18s 192ms/step - accuracy: 0.9623 - loss: 0.1658 - val_accu